# 동적 웹페이지 크롤링
    [다음 뉴스  기사 댓글 가져오기]

## 4가지 기능 미리보기
### 1. 특정 태그 일정 시간 기다리기

<img src="cr_f1.png" width="525" height="122">

### 2. 특정 태그 존재 여부 확인 가능
from selenium.webdriver.common.by  import by
- 해당 태그가 존재하는지 확인
    - 예(By.ID, "alex-area")
- 태그 선택 방법
    - class_name
    - css_selector
    - id
    - name
    - tag_name
    
###  3. 특정 태그 일정 시간 기다리기 기능

<img src="cr_f2.png" width="525" height="122">

### 4. 키보드/ 마우스 동작 자동화하기
from selenium import webdriver

hidden_submenu = driver.find_element_by_css_selector(".nav #submenu1") <br>

actions = webdriver.ActionChains(driver) <br>
actions.click(hidden_submenu) <br>
actions.perform()<br>
또는 <br>
**webdriver.ActionChains(driver).click(hidden_submenu).perform()**


## 웹사이트 열기

In [ ]:
chromedriver = '/usr/local/Cellar/chromedriver' # 각자 chromedriver 가 저장된 경로
driver = webdriver.Chrome(chromedriver)
driver.get(     ) # 괄호 안 원하는 다음 뉴스 링크 (댓글 많은 기사가 예제로 사용하기에 적절)

## Before Crawling

1. 웹페이지 들어가기
2. f12 element '더보기' 소스 확인 (inspector)
- a href="#more" class="link_fold #more" data-tiara-action-name="댓글목록_더보기" data-reactid=".0.0.0.3.3.0"><span data-reactid=".0.0.0.3.3.0.0">더보기</span><span class="img_cmt img_more" data-reactid=".0.0.0.3.3.0.1"></span></a>
- copy - copy selector
    - css selector 이용 가능
- #alex-area > div > div > div > div.cmt_box > div.alex_more > a
3. view page source
4. look up "alex area"

In [ ]:
loop, count = True, 0
# loop = 0
# count = 0

while loop and count < 10:
    # loop and 항상 True
    try:
        element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#alex-area > div > div > div > div.cmt_box > div.alex_more > a')) 
            # 해당 코드가 있는지 동적 페이지 확인
        )
        more_button = driver.find_element_by_css_selector('#alex-area > div > div > div > div.cmt_box > div.alex_more > a')
        webdriver.ActionChains(driver).click(more_button).perform()
        count += 1 # count 10까지만 할 것임
        
        time.sleep(2)
        
    except TimeoutException:
        loop = False
        # 더보기가 늘어날수록 count +=1, 10번 loop 사용
        # count 10 되었을 때 loop 멈춤
        # 10 되기 전에 더보기가 없으면 (댓글이 더이상 없다) exception 실행
        
# crawling code 넣을 공간
    
driver.quit()

## Crawling

In [ ]:
comment_box = driver.find_element_by_css_selector('#cbox_module > div.u_cbox_wrap.u_cbox_ko.u_cbox_type_sort_favorite > div.u_cbox_content_wrap > ul')
comment_list = comment_box.find_elements_by_tag_name('li')
comment_div = comment_box.find_elements_by_css_selector('div')


for num, comment_item in enumerate (comment_list): #넘버링으로 댓글 추출
    print("[", str(num+1), "] ", comment_item.find_element_by_css_selector('div span').text)



<img src="cr_f3.png" width="1400" height="635">

**ul** 전체 댓글 박스 <br>
**li** 각 댓글 박스 <br>
**div p** 박스 안 댓글 텍스트 <br>

## 예시
- 적용1. 추천순 댓글은 전체 댓글 보여주지 않는 현상
    - recent_button variable 지정해줌으로써 최신순 댓글 클릭해주기
- 적용2. 이모티콘만 있는 댓글의 경우 오류 발생하는 현상 수정
    - 댓글 print 할 때  try/  except 함수 사용해서 이모티콘만 있는 댓글은 스킵함

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import time


chromedriver = '/usr/local/Cellar/chromedriver'
driver = webdriver.Chrome(chromedriver)
driver.get('https://news.v.daum.net/v/20200722164549123')

recent_button = driver.find_element_by_css_selector('#alex-area > div > div > div > div.cmt_box > ul.list_category > li:nth-child(3) > button > span')
webdriver.ActionChains(driver).click(recent_button).perform()

loop, count = True, 0
# loop = 0
# count = 0
while loop and count < 10:

    try:
        element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#alex-area > div > div > div > div.cmt_box > div.alex_more > a'))
        )
        more_button = driver.find_element_by_css_selector('#alex-area > div > div > div > div.cmt_box > div.alex_more > a')
        webdriver.ActionChains(driver).click(more_button).perform()
        count += 1 
        
        time.sleep(2)
        
    except TimeoutException:
        loop = False
        
comment_box = driver.find_element_by_css_selector('#alex-area > div > div > div > div.cmt_box > ul.list_comment')
comment_list = comment_box.find_elements_by_tag_name('li')

for num, comment_item in enumerate (comment_list): #각각의 li 추출
    try:
        print("[", str(num+1), "] ", comment_item.find_element_by_css_selector('div p').text)
    except:
        pass
        

driver.quit()